In [1]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from models import SympyTorch, PartialDerivativeCalculator
from parametric_discovery_pinn import ParametricPINN, BurgerPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

feature_names = ['u', 'u_x', 'u_xx']

Loaded from ../parametric_pde_data/parametric_burgers.pkl


In [3]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))

Loaded the pretrained weights


<All keys matched successfully>

In [4]:
expression, _ = build_exp("-0.1872898*sin(t)-1.0238724")
mod = SympyTorch(expressions=[expression])

Please ensure that all coefs are different


In [5]:
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)

In [6]:
xx, tt = dimension_slicing(X_train)
u = model(cat(xx, tt))
gradients_dict(u, xx, tt, feature_names)

{'u': tensor([[-1.9075e-04],
         [ 3.0206e-04],
         [ 5.3166e-04],
         ...,
         [-1.4277e-04],
         [-3.6139e-05],
         [ 7.9684e-04]], grad_fn=<AddmmBackward>),
 'u_x': tensor([[-1.2518e-04],
         [-3.4528e-04],
         [ 1.6754e-03],
         ...,
         [-1.8746e-05],
         [-2.1789e-04],
         [ 1.0137e-03]], grad_fn=<SliceBackward>),
 'u_xx': tensor([[-1.2518e-04],
         [-3.4528e-04],
         [ 1.6754e-03],
         ...,
         [-1.8746e-05],
         [-2.1789e-04],
         [ 1.0137e-03]], grad_fn=<SliceBackward>)}

In [7]:
mod1, variables1 = string2sympytorch("u*u_x")
mod1(gradients_dict(u, xx, tt, sorted(list(map(str, variables1)))))

Please ensure that all coefs are different


tensor([[ 2.3879e-08],
        [-1.0430e-07],
        [ 8.9074e-07],
        ...,
        [ 2.6764e-09],
        [ 7.8742e-09],
        [ 8.0775e-07]], grad_fn=<SqueezeBackward1>)

In [8]:
mod2, variables2 = string2sympytorch("u_xx")
mod2(gradients_dict(u, xx, tt, sorted(list(map(str, variables2)))))

Please ensure that all coefs are different


tensor([[-1.2518e-04],
        [-3.4528e-04],
        [ 1.6754e-03],
        ...,
        [-1.8746e-05],
        [-2.1789e-04],
        [ 1.0137e-03]], grad_fn=<SqueezeBackward1>)

In [9]:
# PartialDerivativeCalculator
# FunctionCalculator

In [10]:
pdc = PartialDerivativeCalculator(["u*u_x", "u_xx"], None)

Please ensure that all coefs are different
Please ensure that all coefs are different


In [11]:
pdc(u, xx, tt)

tensor([[ 2.3879e-08, -1.2518e-04],
        [-1.0430e-07, -3.4528e-04],
        [ 8.9074e-07,  1.6754e-03],
        ...,
        [ 2.6764e-09, -1.8746e-05],
        [ 7.8742e-09, -2.1789e-04],
        [ 8.0775e-07,  1.0137e-03]], grad_fn=<CatBackward>)